In [ ]:
#Libraries
#Helpers

#EDA initial
#Preprocessing
#Eda Final
#Second Preprocessing

#Preprocessing Pipeline
#Model Core Pipeline
#Model Output Pipeline

#Testing


## Helpers

In [19]:

def preprocessor(text):
    text = text.replace('\\r', '').replace('&nbsp', '').replace('\n', '')
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

In [24]:

def coordinates(zipcode):
    zipcode = search.by_zipcode(zipcode)
    community = zipcode.post_office_city
    return community, "{},{}".format(zipcode.lat, zipcode.lng)

def creating_coordinates(df, column):
    df["Community"] = ""
    df["Coordinates"] = "None, None"
    for zipcode in df[column].unique():
        community, coordinate = coordinates(zipcode)
        df.loc[df[column] == zipcode, "Coordinates"] =  str(coordinate)
        df.loc[df[column] == zipcode, "Community"] =  community
    return df

In [305]:
def user_exist(user):
    if len(users.loc[users['UserID'] == user]) == 0:
        return False
    return True

def has_coordinates(user):
    if len(users.loc[users['UserID'] == user, "Coordinates"]) == 0:
        return False
    return True
    


In [138]:
def user_profile(user_id):
    user_profile = np.array(users.loc[users['UserID'] == user_id, ['DegreeType', 'Major']])[0]
    historical_apps = user_history.loc[user_history.UserID == user_id, 'JobTitle']
    user_profile = """
    Degree Type: {}
    Major: {}
    """.format(user_profile[0], user_profile[1])

    return user_profile

def historical_application(user_id):
    historical_apps = apps.loc[apps.UserID == user_id]['JobID']
    content = list()
    for application in historical_apps:
        temp = jobs.loc[jobs.JobID == application, ['Title', 'Description', 'Requirements']]
        if len(temp) != 0:
            content += [temp.Title.values + ". " + temp.Description.values + ". " + temp.Requirements.values]
    return content


In [45]:
class EpochLogger(CallbackAny2Vec):
    '''Callback to log information about training'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))
        self.epoch += 1
        

### Libraries

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import ast 
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
# from nltk.stem.snowball import SnowballStemmer
# from nltk.stem.wordnet import WordNetLemmatizer
# from nltk.corpus import wordnet
# from surprise import Reader, Dataset, SVD, evaluate
from gensim.models import Doc2Vec
import gensim.models.doc2vec
from collections import OrderedDict
import multiprocessing

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.utils import simple_preprocess
from gensim.test.utils import get_tmpfile
from gensim.models.callbacks import CallbackAny2Vec

import pickle

from uszipcode import SearchEngine
from  geopy.distance import distance

import gc
import re

import warnings; warnings.simplefilter('ignore')

#### Checking Cores

In [3]:
assert gensim.models.doc2vec.FAST_VERSION > -1, "SLOW VERSION"

In [4]:
MAX_DISTANCE = 15

#!dir .\Data\*.tsv

In [5]:
folder = 'Data'

apps = pd.read_csv('./'+folder+'/apps.tsv', delimiter='\t',encoding='utf-8')
user_history = pd.read_csv('./'+folder+'/user_history.tsv', delimiter='\t',encoding='utf-8')
jobs = pd.read_csv('./'+folder+'/jobs.tsv', delimiter='\t',encoding='utf-8', error_bad_lines=False)
users = pd.read_csv('./'+folder+'/users.tsv' ,delimiter='\t',encoding='utf-8')
test_users = pd.read_csv('./'+folder+'/test_users.tsv', delimiter='\t',encoding='utf-8')


b'Skipping line 122433: expected 11 fields, saw 12\n'
b'Skipping line 602576: expected 11 fields, saw 12\n'
b'Skipping line 990950: expected 11 fields, saw 12\n'


### EDA

In [6]:
apps.head()


,UserID,WindowID,Split,ApplicationDate,JobID
0,47,1,Train,2012-04-04 15:56:23.537,169528
1,47,1,Train,2012-04-06 01:03:00.003,284009
2,47,1,Train,2012-04-05 02:40:27.753,2121
3,47,1,Train,2012-04-05 02:37:02.673,848187
4,47,1,Train,2012-04-05 22:44:06.653,733748


In [7]:
user_history.head()

,UserID,WindowID,Split,Sequence,JobTitle
0,47,1,Train,1,National Space Communication Programs-Special ...
1,47,1,Train,2,Detention Officer
2,47,1,Train,3,"Passenger Screener, TSA"
3,72,1,Train,1,"Lecturer, Department of Anthropology"
4,72,1,Train,2,Student Assistant


In [8]:
jobs.head()

,JobID,WindowID,Title,Description,Requirements,City,State,Country,Zip5,StartDate,EndDate
0,1,1,Security Engineer/Technical Lead,<p>Security Clearance Required:&nbsp; Top Secr...,<p>SKILL SET</p>\r<p>&nbsp;</p>\r<p>Network Se...,Washington,DC,US,20531,2012-03-07 13:17:01.643,2012-04-06 23:59:59
1,4,1,SAP Business Analyst / WM,<strong>NO Corp. to Corp resumes&nbsp;are bein...,<p><b>WHAT YOU NEED: </b></p>\r<p>Four year co...,Charlotte,NC,US,28217,2012-03-21 02:03:44.137,2012-04-20 23:59:59
2,7,1,P/T HUMAN RESOURCES ASSISTANT,<b> <b> P/T HUMAN RESOURCES ASSISTANT</b> <...,Please refer to the Job Description to view th...,Winter Park,FL,US,32792,2012-03-02 16:36:55.447,2012-04-01 23:59:59
3,8,1,Route Delivery Drivers,CITY BEVERAGES Come to work for the best in th...,Please refer to the Job Description to view th...,Orlando,FL,US,NaN,2012-03-03 09:01:10.077,2012-04-02 23:59:59
4,9,1,Housekeeping,I make sure every part of their day is magica...,Please refer to the Job Description to view th...,Orlando,FL,US,NaN,2012-03-03 09:01:11.88,2012-04-02 23:59:59


In [9]:
users.head()

,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany
0,47,1,Train,Paramount,CA,US,90723,High School,NaN,1999-06-01 00:00:00,3,10.0,Yes,No,0
1,72,1,Train,La Mesa,CA,US,91941,Master's,Anthropology,2011-01-01 00:00:00,10,8.0,Yes,No,0
2,80,1,Train,Williamstown,NJ,US,08094,High School,Not Applicable,1985-06-01 00:00:00,5,11.0,Yes,Yes,5
3,98,1,Train,Astoria,NY,US,11105,Master's,Journalism,2007-05-01 00:00:00,3,3.0,Yes,No,0
4,123,1,Train,Baton Rouge,LA,US,70808,Bachelor's,Agricultural Business,2011-05-01 00:00:00,1,9.0,Yes,No,0


In [10]:
test_users.head()
len(users)

389708

In [11]:
users = users.loc[users.State == 'NY']
len(users)

20590

In [12]:
users = users.sample(frac=0.2, replace=False, random_state=1)
len(users)

4118

#### Subsetting jobs in NY

In [13]:
jobs = jobs.loc[jobs.State == 'NY']
all_jobs = jobs.copy()
#jobs.to_csv("NYjobs.tsv",  sep='\t',encoding='utf-8')

#### Subsetting jobs in Zip

In [14]:
jobs.Zip5 = jobs.Zip5.fillna(0)
jobs.Zip5 = jobs.Zip5.astype(int)
jobs = jobs.loc[jobs['Zip5'] != 0]

In [15]:
len(jobs)

33679

In [16]:

jobs = jobs.sample(frac=0.2, replace=False, random_state=1)
len(jobs)

6736

In [17]:

gc.collect()

0

In [18]:

jobs.groupby(['Zip5', 'City']).size().reset_index(name='Count').sort_values('Count', ascending=False).head()

,Zip5,City,Count
3,10001,New York,630
24,10017,New York,209
414,11747,Melville,140
22,10016,New York,138
46,10036,New York,129


### Preprocessing

### Preprocessing Description and Requirements

In [20]:

jobs['Description'] = jobs['Description'].astype(dtype='str').apply(preprocessor)

In [21]:
jobs['Requirements'] = jobs['Requirements'].astype(dtype='str').apply(preprocessor)

### Creating jobs coordinates

In [22]:
search = SearchEngine(simple_zipcode=True)

In [23]:
#lat, "long"
"""coords_1 = c
coords_2 = (34.1, -118.42)

print (round(distance(coords_1, coords_2).miles, 2))"""
search.by_zipcode("2e").lat

In [25]:
%%timeit
jobs["Community"] = ""
jobs["Coordinates"] = ""
for zipcode in jobs.Zip5.unique():
    community, coordinate = coordinates(zipcode)
    jobs.loc[jobs.Zip5 == zipcode, "Coordinates"] =  str(coordinate)
    jobs.loc[jobs.Zip5 == zipcode, "Community"] =  community

4.12 s ± 96.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
jobs.groupby(['Community']).size().reset_index(name='Count').sort_values('Count', ascending=False).head(20)

,Community,Count
272,"New York, NY",2079
335,"Rochester, NY",329
47,"Buffalo, NY",241
45,"Brooklyn, NY",228
43,"Bronx, NY",165
243,"Melville, NY",141
0,"Albany, NY",109
417,"White Plains, NY",109
374,"Syracuse, NY",91
163,"Hauppauge, NY",75


In [27]:
jobs.groupby(['Community', 'Zip5', 'Coordinates']).size().reset_index(name='Count').sort_values('Count', ascending=False).head(20)

,Community,Zip5,Coordinates,Count
386,"New York, NY",10001,"40.75,-73.99",633
399,"New York, NY",10017,"40.75,-73.973",212
398,"New York, NY",10016,"40.74,-73.97",143
354,"Melville, NY",11747,"40.79,-73.4",141
416,"New York, NY",10036,"40.76,-73.99",134
400,"New York, NY",10018,"40.76,-73.99",118
404,"New York, NY",10022,"40.76,-73.97",97
390,"New York, NY",10005,"40.705,-74.005",90
83,"Brooklyn, NY",11201,"40.7,-73.99",87
401,"New York, NY",10019,"40.77,-73.99",76


### creating user coordinates

In [28]:

users = creating_coordinates(users, 'ZipCode')

In [29]:
users.groupby(['Community']).size().reset_index(name='Count').sort_values('Count', ascending=False).head(20)

,Community,Count
49,"Brooklyn, NY",544
284,"New York, NY",446
46,"Bronx, NY",405
354,"Rochester, NY",200
397,"Staten Island, NY",90
208,"Jamaica, NY",88
458,"Yonkers, NY",62
51,"Buffalo, NY",61
138,"Flushing, NY",38
11,"Astoria, NY",36


### Distance Matrix 

In [30]:
users.head(2)

,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany,Community,Coordinates
200098,248008,4,Train,New York,NY,US,10003,None,Electrical Engineering,NaN,2,15.0,Yes,No,0,"New York, NY","40.73,-73.99"
286529,1184529,5,Train,New York,NY,US,10035,High School,Not Applicable,2005-01-01 00:00:00,3,7.0,NaN,No,0,"New York, NY","40.8,-73.93"


In [31]:
jobs.head(2)

,JobID,WindowID,Title,Description,Requirements,City,State,Country,Zip5,StartDate,EndDate,Community,Coordinates
322649,268634,2,On-Call Office Services Associate (Scanning/Ma...,pitney bowes management services pbms provides...,the ideal candidate must be able to demonstrat...,Armonk,NY,US,10504,2012-04-16 14:03:48.903,2012-05-15 23:59:59,"Armonk, NY","41.13,-73.69"
1073169,936381,7,"Coordinator, Corporate Marketing",gannett corporate marketing coordinator corpo...,,New York,NY,US,10001,2012-05-24 21:05:10.253,2012-06-23 23:59:00,"New York, NY","40.75,-73.99"


In [32]:
apps.head(2)

,UserID,WindowID,Split,ApplicationDate,JobID
0,47,1,Train,2012-04-04 15:56:23.537,169528
1,47,1,Train,2012-04-06 01:03:00.003,284009


In [33]:
user_history.head(2)

,UserID,WindowID,Split,Sequence,JobTitle
0,47,1,Train,1,National Space Communication Programs-Special ...
1,47,1,Train,2,Detention Officer


### Subset most applied jobs by title

### Looking similar jobs

In [34]:
current_app_intercep = np.intersect1d(users.UserID.unique(), apps.UserID.unique())
historical_app_intercep = np.intersect1d(users.UserID.unique(), user_history.UserID.unique())
total_users = users.UserID.unique()
with_info =  set(current_app_intercep.tolist() + historical_app_intercep.tolist())

current_app = 1-len(current_app_intercep )/len(total_users )
historical_app = 1-len(historical_app_intercep)/ len(total_users )
cold_star_p = len(with_info)/len(total_users )

print(current_app, historical_app, cold_star_p)


0.1733851384167071 0.03302574065080133 0.994900437105391


### Looks for most applied jobs id

### Recomender top k liked

In [39]:
unique = 0.3 #update name
top = 20

def ranking_by_popularity(top):
    
    popular_jobs = user_history.groupby(
    ['JobTitle']).size().reset_index(
    name='Count').sort_values('Count', ascending=False)
    ranking =  dict()
    top_i = 0
    
    while True:
        job_title = popular_jobs['JobTitle'].iloc[top_i]
        jobs_list = jobs.loc[jobs['Title'] == job_title, ['JobID']]['JobID'].unique().tolist()

        if len(jobs_list) > 1:
            ranking[job_title] = jobs_list

        if len(ranking) == top:
            break

        top_i +=1
    return ranking



In [142]:

def recommender_popular_jobs(user_id,  top):
    
    recommended_popular_jobs = dict()
    c1 = users.loc[users['UserID'] == user_id, 'Coordinates']
    
    if user_exist(user_id) and has_coordinates(user_id):
        for title, jobs_list in ranking_popular.items():
            
            distances = dict()
            
            for job in jobs_list:
                
                c2 = jobs.loc[jobs['JobID'] == job, 'Coordinates']
                if c2.iloc[0].split(',')[0] == 'None':
                    continue
                distances[job] = round(distance(c1, c2).miles, 2)
            
            distances = sorted(distances.items(), key=lambda kv: kv[1])
            closest = distances[0]
            if closest[1] >= MAX_DISTANCE:
                continue
            recommended_popular_jobs[title] = (closest[0], closest[1])
            
    else:
        for title, jobs_list in ranking_popular.items():
            recommended_popular_jobs[title] = jobs_list[0]
            
    return recommended_popular_jobs

ranking_popular = ranking_by_popularity(top)
recommender_popular_jobs(98,  top)

{'Customer Service Representative': 1015707,
 'Cashier': 668223,
 'Administrative Assistant': 617073,
 'Sales Associate': 407797,
 'Assistant Manager': 317522,
 'Office Manager': 203695,
 'Manager': 1097462,
 'Receptionist': 518270,
 'Customer Service': 119569,
 'Store Manager': 1075099,
 'General Manager': 286330,
 'Sales Representative': 6460,
 'Medical Assistant': 778080,
 'Project Manager': 127738,
 'Account Executive': 865392,
 'Account Manager': 158909,
 'Sales Manager': 513121,
 'Executive Assistant': 166751,
 'Sales': 1076762,
 'Teller': 376812}

### Content Based title x profile



In [41]:
jobs['profile'] = jobs['Title'].astype(str)  +  '. ' + jobs['Requirements'].astype(str) +  '. ' + jobs['Description'].astype(str) 

In [316]:
%%timeit

users['profile'] =  (users['Major'].astype(str) +  " " ) + (users['DegreeType'].astype(str) + " ")
#+ " Years of Experience " + users['TotalYearsExperience'].astype(str)
#+ str(p.number_to_words(users["TotalYearsExperience"])) + " Years of Experience " + 


2.57 ms ± 361 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


##### Garbage words

In [312]:
users['profile'] = users['profile'].str.replace('None.', ' ')
users['profile'] = users['profile'].str.replace('Not Applicable', ' ')
users['profile'] = users['profile'].str.replace('nan', ' ') 

users['profile'].iloc[1]

'  High School '

In [313]:

users.loc[users['UserID'] == 4365]

,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany,Community,Coordinates,profile


In [57]:
apps.head()

,UserID,WindowID,Split,ApplicationDate,JobID
0,47,1,Train,2012-04-04 15:56:23.537,169528
1,47,1,Train,2012-04-06 01:03:00.003,284009
2,47,1,Train,2012-04-05 02:40:27.753,2121
3,47,1,Train,2012-04-05 02:37:02.673,848187
4,47,1,Train,2012-04-05 22:44:06.653,733748


In [220]:
%%time
def similarities_nlp_model( model_name = "jobs_doc2vec_model",
                 mapping_name = "jobID_mapping.p", max_epochs = 100,
                 alpha = 0.025):
    
    document = list()
    jobID_mapping = dict()
        
    for i, token in enumerate(jobs['profile']):
        value = jobs.iloc[i]["JobID"]
        tokens = TaggedDocument(simple_preprocess(token), [i])
        document.append(tokens)
        jobID_mapping[i] = value

    epoch_logger = EpochLogger()
    model = Doc2Vec(size = 20, alpha=alpha, 
                    min_alpha=0.00025, min_count=2,
                    callbacks=[epoch_logger], dm =1, workers=8, window=2)
    
    model.build_vocab(document)
    print(model.corpus_count)
    for epoch in range(max_epochs):
        print('iteration {0}'.format(epoch))
        model.train(document, 
                    total_examples=model.corpus_count, 
                    epochs=model.iter)
        # decrease the learning rate
        model.alpha -= 0.0002
        # fix the learning rate, no decay
        model.min_alpha = model.alpha

    # create a dictionary
    pickle.dump(model, open(model_name, "wb")) 
    pickle.dump(jobID_mapping, open(mapping_name, "wb")) 

# -------------------------------------------------------------
# Load the dictionary back from the pickle file.
    return model, jobID_mapping

model, jobID_mapping = similarities_nlp_model()


6736
iteration 0
Epoch #0 start
Epoch #1 start
Epoch #2 start
Epoch #3 start
Epoch #4 start
iteration 1
Epoch #5 start
Epoch #6 start
Epoch #7 start
Epoch #8 start
Epoch #9 start
iteration 2
Epoch #10 start
Epoch #11 start
Epoch #12 start
Epoch #13 start
Epoch #14 start
iteration 3
Epoch #15 start
Epoch #16 start
Epoch #17 start
Epoch #18 start
Epoch #19 start
iteration 4
Epoch #20 start
Epoch #21 start
Epoch #22 start
Epoch #23 start
Epoch #24 start
iteration 5
Epoch #25 start
Epoch #26 start
Epoch #27 start
Epoch #28 start
Epoch #29 start
iteration 6
Epoch #30 start
Epoch #31 start
Epoch #32 start
Epoch #33 start
Epoch #34 start
iteration 7
Epoch #35 start
Epoch #36 start
Epoch #37 start
Epoch #38 start
Epoch #39 start
iteration 8
Epoch #40 start
Epoch #41 start
Epoch #42 start
Epoch #43 start
Epoch #44 start
iteration 9
Epoch #45 start
Epoch #46 start
Epoch #47 start
Epoch #48 start
Epoch #49 start
iteration 10
Epoch #50 start
Epoch #51 start
Epoch #52 start
Epoch #53 start
Epoch #5

Epoch #424 start
iteration 85
Epoch #425 start
Epoch #426 start
Epoch #427 start
Epoch #428 start
Epoch #429 start
iteration 86
Epoch #430 start
Epoch #431 start
Epoch #432 start
Epoch #433 start
Epoch #434 start
iteration 87
Epoch #435 start
Epoch #436 start
Epoch #437 start
Epoch #438 start
Epoch #439 start
iteration 88
Epoch #440 start
Epoch #441 start
Epoch #442 start
Epoch #443 start
Epoch #444 start
iteration 89
Epoch #445 start
Epoch #446 start
Epoch #447 start
Epoch #448 start
Epoch #449 start
iteration 90
Epoch #450 start
Epoch #451 start
Epoch #452 start
Epoch #453 start
Epoch #454 start
iteration 91
Epoch #455 start
Epoch #456 start
Epoch #457 start
Epoch #458 start
Epoch #459 start
iteration 92
Epoch #460 start
Epoch #461 start
Epoch #462 start
Epoch #463 start
Epoch #464 start
iteration 93
Epoch #465 start
Epoch #466 start
Epoch #467 start
Epoch #468 start
Epoch #469 start
iteration 94
Epoch #470 start
Epoch #471 start
Epoch #472 start
Epoch #473 start
Epoch #474 start
ite

In [180]:
#If user has coordinates
#if user has info then similarties if not popularity ranker
jobID_mapping = pickle.load(open("jobID_mapping.p", "rb"))
model  = pickle.load(open("jobs_doc2vec_model", "rb"))

def content_distance_based_recommender(user_id, jobID_mapping = jobID_mapping, model =  model, top = 10):
    #As infer_vector produce stochastics result I made a for to save the best list
    user_profile = np.array(users.loc[users['UserID'] == user_id, 'profile'])[0]

    historical_apps = historical_application(user_id)
    for application in historical_apps:
        user_profile += ". " + (str(application[0]) + " ")
    user_profile = simple_preprocess(user_profile)
    
    best = 0
    tops =pd.DataFrame(index = range(top), columns = ['JobID', 'Title', 'Distance', 'Description', 'Requirements'])
    c1 = users.loc[users['UserID'] == user_id, 'Coordinates']
    job_distance_list = list()
    
    for i in range (1):
        inferred_vector = model.infer_vector(user_profile)
        sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
        sum_results = 0
        count = 0
        total_recom = 0
        job_distance_list = list()
        
        while True:
            job_id = jobID_mapping[sims[count][0]]
            c2 = jobs.loc[jobs['JobID'] == jobID_mapping[sims[count][0]], 'Coordinates']
            count +=1
            if len(c2) == 0:
                #print("Empty", sims[count][0])
                continue
            
            if c2.iloc[0].split(',')[0] == 'None':
                #print('None', sims[count][0])
                continue
                
            job_distance = round(distance(c1, c2).miles, 2)
            
            if job_distance >= MAX_DISTANCE:
                #print('Distance', job_distance,  sims[count][0])
                continue

            sum_results+=sims[count][1]
            total_recom +=1
            job_distance_list.append(job_distance)
            if total_recom == top:
                break
        
        ##Best simulation
        if sum_results > best:
            best = sum_results
            best_sim = sims
            job_distance_list2 = job_distance_list
            
    for i in range(top):
        recomendation = jobID_mapping[best_sim[i][0]]
        tops.iloc[i]['JobID', 'Title',  'Description', 'Requirements'] = np.array(jobs.loc[jobs['JobID'] == recomendation][['JobID', 'Title', 
                                                               'Description', 'Requirements']])[0]
        tops.iloc[i]['Distance'] = job_distance_list2[i]
    
    return tops 
 
        

In [255]:
users.tail()

,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany,Community,Coordinates,profile
312978,525060,6,Train,Bronx,NY,US,10458,High School,Not Applicable,1997-01-01 00:00:00,4,11.0,No,No,0,"Bronx, NY","40.86,-73.89",High S...
70860,1357269,1,Train,Bronx,NY,US,10451,Bachelor's,Forensic Accounting,2011-09-01 00:00:00,5,4.0,No,No,0,"Bronx, NY","40.82,-73.92",Forensic Accounting Forensic Accounting Forens...
350515,306236,7,Train,Port Washington,NY,US,11050,High School,Not Applicable,1992-01-01 00:00:00,4,20.0,Yes,Yes,30,"Port Washington, NY","40.84,-73.69",High S...
255382,301152,5,Train,Bronx,NY,US,10451,None,NaN,NaN,9,3.0,Yes,No,0,"Bronx, NY","40.82,-73.92",nan
119559,1092242,2,Train,Bronx,NY,US,10467,Bachelor's,Psychology,2011-09-01 00:00:00,1,12.0,No,No,0,"Bronx, NY","40.88,-73.87",Psychology Psychology Psychology Psychology Ps...


In [317]:
user_id = 	306236		
top_recommendations = content_distance_based_recommender(user_id, top=10)
print("USER PROFILE: ", user_profile(user_id))
print("HISTORICAL APPLICATION CONTENT:\n\n", historical_application(user_id))
users[users.UserID == user_id].profile.values


USER PROFILE:  
    Degree Type: High School
    Major: Not Applicable
    
HISTORICAL APPLICATION CONTENT:

 []


array(['Not Applicable High School '], dtype=object)

In [318]:
top_recommendations

,JobID,Title,Distance,Description,Requirements
0,958538,Retail Wireless Consultant,11.74,job functions duties responsibilities,
1,79739,Retail Wireless Consultant,13.1,job functions duties responsibilities,
2,958523,Retail Wireless Consultant,12.27,job functions duties responsibilities,
3,1112854,sdfsdf,9.8,test,test
4,380021,Retail Wireless Consultant,12.43,job functions duties responsibilities,
5,679983,Retail Wireless Consultant,9.8,job functions duties responsibilities,
6,380034,Retail Wireless Consultant,12.46,job functions duties responsibilities,
7,549639,ADMINISTRATIVE/CLERICAL,11.79,colating,nan
8,849368,ADMINISTRATIVE/CLERICAL,13,colating,nan
9,958641,Retail Wireless Consultant - Part Time,12.02,job functions duties responsibilities,


In [323]:
def hybrid_recommender(user_id, top=10):
    
    if has_coordinates(user_id) and len(historical_application(user_id)) > 0:
        print("Content")
        return content_distance_based_recommender(user_id, top = top)
    else:
        print("Cold Star")
        return [], recommender_popular_jobs(user_id,  top = top)
    


In [324]:
# ColdStar 301152
hybrid_recommender(306236)


Cold Star


([],
 {'Customer Service Representative': (819507, 13.74),
  'Cashier': (668223, 12.43),
  'Administrative Assistant': (147340, 4.94),
  'Sales Associate': (407797, 0.0),
  'Assistant Manager': (273546, 8.03),
  'Office Manager': (882404, 7.87),
  'Manager': (1097462, 9.68),
  'Receptionist': (518270, 6.93),
  'Store Manager': (911774, 6.92),
  'Project Manager': (723794, 12.25),
  'Account Executive': (227425, 14.82),
  'Account Manager': (283616, 8.35),
  'Executive Assistant': (766175, 13.61),
  'Sales': (544746, 12.27),
  'Teller': (259223, 7.98)})